In [1]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from mlxtend.classifier import StackingCVClassifier, StackingClassifier

from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMClassifier

import numpy as np

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
RANDOM_SEED = 2

In [4]:
lr_pipe = Pipeline([('prep', StandardScaler()),
                    ('lr', LogisticRegression())])
knn_pipe = Pipeline([('prep', StandardScaler()),
                     ('knn', KNeighborsClassifier())])
rf = RandomForestClassifier(random_state=RANDOM_SEED)

In [5]:
np.random.seed(RANDOM_SEED)
sclf1 = StackingCVClassifier(classifiers=[lr_pipe, knn_pipe, rf], 
                             cv=3,
                             use_features_in_secondary=True, 
#                              use_probas=True,
                             meta_classifier=LogisticRegression(random_state=RANDOM_SEED))

sclf2 = StackingCVClassifier(classifiers=[lr_pipe, knn_pipe, rf], 
                             cv=3,
                             use_features_in_secondary=True, 
#                              use_probas=True,
                             meta_classifier=RandomForestClassifier(random_state=RANDOM_SEED))

top_sclf = StackingCVClassifier(classifiers=[sclf1, sclf2], 
                                cv=3,
                                use_features_in_secondary=True, 
#                              use_probas=True,
                                 meta_classifier=RandomForestClassifier(random_state=RANDOM_SEED))

top_sclf2 = StackingCVClassifier(classifiers=[sclf1, sclf2], 
                                 cv=3,
                                 use_features_in_secondary=True,  
#                              use_probas=True,
                                 meta_classifier=LGBMClassifier(seed=RANDOM_SEED))

In [6]:
top_sclf.get_params().keys()

dict_keys(['stackingcvclassifier-1', 'stackingcvclassifier-2', 'stackingcvclassifier-1__pipeline-1', 'stackingcvclassifier-1__pipeline-2', 'stackingcvclassifier-1__randomforestclassifier', 'stackingcvclassifier-1__pipeline-1__steps', 'stackingcvclassifier-1__pipeline-1__prep', 'stackingcvclassifier-1__pipeline-1__lr', 'stackingcvclassifier-1__pipeline-1__prep__copy', 'stackingcvclassifier-1__pipeline-1__prep__with_mean', 'stackingcvclassifier-1__pipeline-1__prep__with_std', 'stackingcvclassifier-1__pipeline-1__lr__C', 'stackingcvclassifier-1__pipeline-1__lr__class_weight', 'stackingcvclassifier-1__pipeline-1__lr__dual', 'stackingcvclassifier-1__pipeline-1__lr__fit_intercept', 'stackingcvclassifier-1__pipeline-1__lr__intercept_scaling', 'stackingcvclassifier-1__pipeline-1__lr__max_iter', 'stackingcvclassifier-1__pipeline-1__lr__multi_class', 'stackingcvclassifier-1__pipeline-1__lr__n_jobs', 'stackingcvclassifier-1__pipeline-1__lr__penalty', 'stackingcvclassifier-1__pipeline-1__lr__rando

In [7]:
top_sclf2.get_params().keys()

dict_keys(['stackingcvclassifier-1', 'stackingcvclassifier-2', 'stackingcvclassifier-1__pipeline-1', 'stackingcvclassifier-1__pipeline-2', 'stackingcvclassifier-1__randomforestclassifier', 'stackingcvclassifier-1__pipeline-1__steps', 'stackingcvclassifier-1__pipeline-1__prep', 'stackingcvclassifier-1__pipeline-1__lr', 'stackingcvclassifier-1__pipeline-1__prep__copy', 'stackingcvclassifier-1__pipeline-1__prep__with_mean', 'stackingcvclassifier-1__pipeline-1__prep__with_std', 'stackingcvclassifier-1__pipeline-1__lr__C', 'stackingcvclassifier-1__pipeline-1__lr__class_weight', 'stackingcvclassifier-1__pipeline-1__lr__dual', 'stackingcvclassifier-1__pipeline-1__lr__fit_intercept', 'stackingcvclassifier-1__pipeline-1__lr__intercept_scaling', 'stackingcvclassifier-1__pipeline-1__lr__max_iter', 'stackingcvclassifier-1__pipeline-1__lr__multi_class', 'stackingcvclassifier-1__pipeline-1__lr__n_jobs', 'stackingcvclassifier-1__pipeline-1__lr__penalty', 'stackingcvclassifier-1__pipeline-1__lr__rando

In [8]:
# instantiate pipeline to select model
pipe = Pipeline([('model', LGBMClassifier())])

In [9]:
top_sclf_search = {'model': Categorical([top_sclf]),
                   'model__stackingcvclassifier-1__pipeline-1__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
                   'model__stackingcvclassifier-1__pipeline-1__lr__penalty': Categorical(['l1', 'l2']),
                   'model__stackingcvclassifier-1__pipeline-2__knn__n_neighbors': Integer(1, 20),
                   'model__stackingcvclassifier-1__randomforestclassifier__max_features': Integer(1, 4),
                   'model__stackingcvclassifier-1__meta-logisticregression__C': Real(1e-6, 1e6, prior='log-uniform'),
                   'model__stackingcvclassifier-1__meta-logisticregression__penalty': Categorical(['l1', 'l2']),

                   'model__stackingcvclassifier-2__pipeline-1__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
                   'model__stackingcvclassifier-2__pipeline-1__lr__penalty': Categorical(['l1', 'l2']),
                   'model__stackingcvclassifier-2__pipeline-2__knn__n_neighbors': Integer(1, 20),
                   'model__stackingcvclassifier-2__randomforestclassifier__max_features': Integer(1, 4),
                   'model__stackingcvclassifier-2__meta-randomforestclassifier__max_features': Integer(1, 4),

                   'model__meta-randomforestclassifier__max_features': Integer(1, 4)
                 }

top_sclf_search2 = {'model': Categorical([top_sclf2]),
                    'model__stackingcvclassifier-1__pipeline-1__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
                    'model__stackingcvclassifier-1__pipeline-1__lr__penalty': Categorical(['l1', 'l2']),
                    'model__stackingcvclassifier-1__pipeline-2__knn__n_neighbors': Integer(1, 20),
                    'model__stackingcvclassifier-1__randomforestclassifier__max_features': Integer(1, 4),
                    'model__stackingcvclassifier-1__meta-logisticregression__C': Real(1e-6, 1e6, prior='log-uniform'),
                    'model__stackingcvclassifier-1__meta-logisticregression__penalty': Categorical(['l1', 'l2']),

                    'model__stackingcvclassifier-2__pipeline-1__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
                    'model__stackingcvclassifier-2__pipeline-1__lr__penalty': Categorical(['l1', 'l2']),
                    'model__stackingcvclassifier-2__pipeline-2__knn__n_neighbors': Integer(1, 20),
                    'model__stackingcvclassifier-2__randomforestclassifier__max_features': Integer(1, 4),
                    'model__stackingcvclassifier-2__meta-randomforestclassifier__max_features': Integer(1, 4),

                    'model__meta-lgbmclassifier__num_leaves': Integer(2, 100)
                 }

In [10]:
opt = BayesSearchCV(pipe, search_spaces=[(top_sclf_search, 50),
                                         (top_sclf_search2, 50)], 
                    n_jobs=-1, random_state=RANDOM_SEED,cv=5)

In [11]:
opt.fit(X, y)

/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


In [12]:
opt.best_estimator_

Pipeline(steps=[('model', StackingCVClassifier(classifiers=[StackingCVClassifier(classifiers=[Pipeline(steps=[('prep', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=9.9999999999999995e-07, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=...uffle=True, stratify=True, use_features_in_secondary=True,
           use_probas=False, verbose=0))])

In [18]:
opt.best_params_

{'model': StackingCVClassifier(classifiers=[StackingCVClassifier(classifiers=[Pipeline(steps=[('prep', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=9.9999999999999995e-07, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class...huffle=True, stratify=True, use_features_in_secondary=True,
            use_probas=False, verbose=0)],
            cv=3,
            meta_classifier=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features=1, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10, n_jobs=1, oob_score=False, random_state=2,
             verbose=0, warm_start=False),
            shuffle=True, stratify=True, use_features_in_secondary=True,
            use_probas=False, verbose=0),
 'model__meta-rando

## NOTE cv_results seem to be messed up

The cv_results seem to be a bit funky. One I get an error when I try to just pass in the cv_results to create a dataframe.  Trying the hack from SO that I found doesn't throw an error but results in messed up column values. For example even though the best estimator has a random forest top level meta-estimator, there is a non-null value for the number of leaves for a LightGBM classifier in that row.

In [34]:
import pandas as pd
cv_results = pd.DataFrame.from_dict(opt.cv_results_, orient='index').transpose()

In [35]:
cv_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 34 columns):
split0_test_score                                                                 112 non-null object
split1_test_score                                                                 112 non-null object
split2_test_score                                                                 112 non-null object
split3_test_score                                                                 112 non-null object
split4_test_score                                                                 112 non-null object
mean_test_score                                                                   112 non-null object
std_test_score                                                                    112 non-null object
rank_test_score                                                                   112 non-null object
split0_train_score                                                             

In [36]:
pd.options.display.max_columns = None
cv_results.sort_values('mean_test_score', ascending=False)

split0_test_score split1_test_score split2_test_score split3_test_score  \
36           0.966667          0.966667          0.933333                 1   
12           0.966667          0.966667          0.933333          0.966667   
16           0.966667          0.966667          0.933333          0.966667   
41           0.966667          0.966667          0.933333          0.966667   
31           0.966667          0.966667          0.933333          0.966667   
7            0.966667          0.966667          0.933333          0.966667   
24           0.966667          0.966667          0.933333          0.966667   
101                 1                 1          0.933333               0.9   
19           0.966667          0.966667          0.966667               0.9   
53           0.966667          0.966667          0.933333          0.933333   
52           0.966667          0.966667          0.933333          0.933333   
46           0.966667          0.966667          0.966667               0.9   
26           0.966667          0.966667          0.933333          0.933333   
39           0.966667          0.966667               0.9          0.966667   
30           0.966667          0.966667               0.9          0.966667   
32           0.966667          0.966667          0.933333          0.933333   
40           0.966667          0.966667          0.933333          0.933333   
21           0.966667          0.966667          0.933333          0.933333   
38           0.966667          0.966667          0.933333          0.933333   
18           0.966667          0.966667          0.933333          0.933333   
5            0.966667          0.966667          0.933333          0.933333   
17           0.966667          0.966667          0.933333          0.933333   
3            0.966667          0.966667          0.933333          0.933333   
13           0.966667          0.966667          0.933333          0.933333   
11           0.966667          0.966667               0.9          0.966667   
97                  1                 1          0.866667          0.933333   
96                  1          0.966667          0.933333               0.9   
6            0.966667          0.966667               0.9          0.966667   
73           0.966667          0.966667          0.866667          0.966667   
95           0.966667          0.966667          0.866667          0.966667   
..                ...               ...               ...               ...   
69           0.933333          0.966667          0.866667               0.9   
109          0.933333          0.966667               0.9          0.866667   
61           0.933333          0.966667          0.866667               0.9   
98           0.966667          0.966667          0.866667          0.866667   
66           0.933333          0.966667          0.866667               0.9   
77           0.933333          0.966667          0.866667               0.9   
93           0.933333          0.966667          0.866667               0.9   
68           0.933333                 1          0.866667          0.866667   
84           0.933333          0.966667          0.866667               0.9   
86           0.933333          0.966667               0.9          0.866667   
71           0.933333          0.966667               0.9          0.866667   
90           0.933333          0.966667          0.866667          0.866667   
110          0.933333          0.966667          0.866667          0.866667   
72           0.933333          0.966667          0.866667          0.866667   
80           0.933333          0.966667          0.866667          0.866667   
81           0.933333          0.966667          0.866667          0.866667   
82           0.933333          0.966667          0.866667          0.866667   
58           0.933333          0.966667          0.866667          0.866667   
60           0.933333          0.966667          0.866

In [39]:
sum(cv_results.param_model.str.contains('meta_classifier=LGBMClassifier'))

nan

In [40]:
sum(rf)

nan

In [47]:
model_type = cv_results.param_model[0].meta_classifier

In [54]:
cv_results['top_stack'] = cv_results.param_model.apply(lambda x: str(x.meta_classifier).split('(')[0])

In [55]:
rf_cv_results = cv_results.loc[cv_results.top_stack=='RandomForestClassifier', :]
lgbm_cv_results = cv_results.loc[cv_results.top_stack!='RandomForestClassifier', :]

In [56]:
rf_cv_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 35 columns):
split0_test_score                                                                 56 non-null object
split1_test_score                                                                 56 non-null object
split2_test_score                                                                 56 non-null object
split3_test_score                                                                 56 non-null object
split4_test_score                                                                 56 non-null object
mean_test_score                                                                   56 non-null object
std_test_score                                                                    56 non-null object
rank_test_score                                                                   56 non-null object
split0_train_score                                                                56 non-

In [57]:
lgbm_cv_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 56 to 111
Data columns (total 35 columns):
split0_test_score                                                                 56 non-null object
split1_test_score                                                                 56 non-null object
split2_test_score                                                                 56 non-null object
split3_test_score                                                                 56 non-null object
split4_test_score                                                                 56 non-null object
mean_test_score                                                                   56 non-null object
std_test_score                                                                    56 non-null object
rank_test_score                                                                   56 non-null object
split0_train_score                                                                56 no

In [60]:
lgbm_cv_results.sort_values('mean_test_score', ascending=False, inplace=True)
lgbm_cv_results.head()

/home/savvas-mint/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


split0_test_score split1_test_score split2_test_score split3_test_score  \
101                 1                 1          0.933333               0.9   
96                  1          0.966667          0.933333               0.9   
97                  1                 1          0.866667          0.933333   
73           0.966667          0.966667          0.866667          0.966667   
95           0.966667          0.966667          0.866667          0.966667   

    split4_test_score mean_test_score std_test_score rank_test_score  \
101                 1        0.966667      0.0421637               1   
96                  1            0.96       0.038873               2   
97                  1            0.96      0.0533333               2   
73                  1        0.953333      0.0452155               1   
95                  1        0.953333      0.0452155               1   

    split0_train_score split1_train_score split2_train_score  \
101               0.95               0.95           0.991667   
96            0.966667               0.95           0.958333   
97                0.95               0.95           0.958333   
73            0.966667           0.958333              0.975   
95            0.941667           0.966667              0.975   

    split3_train_score split4_train_score mean_train_score std_train_score  \
101           0.966667           0.966667            0.965       0.0152753   
96                   1           0.966667         0.968333       0.0169967   
97                   1           0.941667             0.96       0.0206828   
73            0.966667           0.941667         0.961667       0.0113039   
95            0.983333           0.966667         0.966667       0.0139443   

    mean_fit_time std_fit_time mean_score_time std_score_time  \
101       1.75467     0.240052       0.0344334      0.0181626   
96        1.38806     0.155343       0.0587132      0.0202172   
97        1.66076        0.235       0.0594453      0.0361461   
73        1.27374     0.153066          0.0523       0.027013   
95        1.68486     0.211843       0.0455001      0.0193287   

                                           param_model  \
101  StackingCVClassifier(classifiers=[StackingCVCl...   
96   StackingCVClassifier(classifiers=[StackingCVCl...   
97   StackingCVClassifier(classifiers=[StackingCVCl...   
73   StackingCVClassifier(classifiers=[StackingCVCl...   
95   StackingCVClassifier(classifiers=[StackingCVCl...   

    param_model__meta-randomforestclassifier__max_features  \
101                                               None       
96                                                None       
97                                                None       
73                                                None       
95                                                None       

    param_model__stackingcvclassifier-1__meta-logisticregression__C  \
101                                             569902                
96                                               1e+06                
97                                               1e+06                
73                                               1e+06                
95                                               1e+06                

    param_model__stackingcvclassifier-1__meta-logisticregression__penalty  \
101                                                 l2                      
96                                                  l1                      
97                                                  l1                      
73                                                  l2                      
95                                                  l2                      

    param_model__stackingcvclassifier-1__pipeline-1__lr__C  \
101                                        5.87864e-06       
96                                               1e+06       
97                                               1e+0

In [80]:
lgbm_cv_results.params[101]

{'model': StackingCVClassifier(classifiers=[StackingCVClassifier(classifiers=[Pipeline(steps=[('prep', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=102442.73115096195, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ov...huffle=True, stratify=True, use_features_in_secondary=True,
            use_probas=False, verbose=0)],
            cv=3,
            meta_classifier=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
         max_bin=255, max_depth=-1, min_child_samples=10,
         min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=-1,
         num_leaves=80, objective='binary', reg_alpha=0, reg_lambda=0,
         seed=2, silent=True, subsample=1, subsample_for_bin=50000,
         subsample_freq=1),
            shuffle=True, stratify=True, use_features_in_secondary=True,
            use_probas=False, verbose=0),
 'model__meta-lgbmclass

Looks like the model parameters match the first row above

In [83]:
# now set the proper num_leaves value
nl = lgbm_cv_results.params.apply(lambda x: 
                                  x['model__meta-lgbmclassifier__num_leaves'])
lgbm_cv_results['param_model__meta-lgbmclassifier__num_leaves'] = nl

/home/savvas-mint/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [84]:
lgbm_cv_results.head()

split0_test_score split1_test_score split2_test_score split3_test_score  \
101                 1                 1          0.933333               0.9   
96                  1          0.966667          0.933333               0.9   
97                  1                 1          0.866667          0.933333   
73           0.966667          0.966667          0.866667          0.966667   
95           0.966667          0.966667          0.866667          0.966667   

    split4_test_score mean_test_score std_test_score rank_test_score  \
101                 1        0.966667      0.0421637               1   
96                  1            0.96       0.038873               2   
97                  1            0.96      0.0533333               2   
73                  1        0.953333      0.0452155               1   
95                  1        0.953333      0.0452155               1   

    split0_train_score split1_train_score split2_train_score  \
101               0.95               0.95           0.991667   
96            0.966667               0.95           0.958333   
97                0.95               0.95           0.958333   
73            0.966667           0.958333              0.975   
95            0.941667           0.966667              0.975   

    split3_train_score split4_train_score mean_train_score std_train_score  \
101           0.966667           0.966667            0.965       0.0152753   
96                   1           0.966667         0.968333       0.0169967   
97                   1           0.941667             0.96       0.0206828   
73            0.966667           0.941667         0.961667       0.0113039   
95            0.983333           0.966667         0.966667       0.0139443   

    mean_fit_time std_fit_time mean_score_time std_score_time  \
101       1.75467     0.240052       0.0344334      0.0181626   
96        1.38806     0.155343       0.0587132      0.0202172   
97        1.66076        0.235       0.0594453      0.0361461   
73        1.27374     0.153066          0.0523       0.027013   
95        1.68486     0.211843       0.0455001      0.0193287   

                                           param_model  \
101  StackingCVClassifier(classifiers=[StackingCVCl...   
96   StackingCVClassifier(classifiers=[StackingCVCl...   
97   StackingCVClassifier(classifiers=[StackingCVCl...   
73   StackingCVClassifier(classifiers=[StackingCVCl...   
95   StackingCVClassifier(classifiers=[StackingCVCl...   

    param_model__meta-randomforestclassifier__max_features  \
101                                               None       
96                                                None       
97                                                None       
73                                                None       
95                                                None       

    param_model__stackingcvclassifier-1__meta-logisticregression__C  \
101                                             569902                
96                                               1e+06                
97                                               1e+06                
73                                               1e+06                
95                                               1e+06                

    param_model__stackingcvclassifier-1__meta-logisticregression__penalty  \
101                                                 l2                      
96                                                  l1                      
97                                                  l1                      
73                                                  l2                      
95                                                  l2                      

    param_model__stackingcvclassifier-1__pipeline-1__lr__C  \
101                                        5.87864e-06       
96                                               1e+06       
97                                               1e+0